In [44]:
import json
import glob

import numpy as np
import pandas as pd

from qiskit import transpile
from qiskit import execute, QuantumRegister, ClassicalRegister, Aer
from qiskit.providers.fake_provider import FakeLima
from qiskit.primitives import Estimator
from qiskit.circuit.random import random_circuit

import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.nn.functional import dropout

from torch_geometric.nn import GCNConv, global_mean_pool, Linear, ChebConv, SAGEConv
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader

from tqdm.notebook import tqdm_notebook
import matplotlib.pyplot as plt
import seaborn as sns

from blackwater.data.loaders.exp_val import CircuitGraphExpValMitigationDataset
from blackwater.data.generators.exp_val import exp_value_generator
from blackwater.data.utils import generate_random_pauli_sum_op
from blackwater.library.ngem.estimator import ngem

from qiskit.quantum_info import random_clifford

import random
from qiskit.circuit.library import HGate, SdgGate
from qiskit.circuit import ClassicalRegister

from blackwater.data.utils import (
    generate_random_pauli_sum_op,
    create_estimator_meas_data,
    circuit_to_graph_data_json,
    get_backend_properties_v1,
    encode_pauli_sum_op,
    create_meas_data_from_estimators
)
from blackwater.data.generators.exp_val import ExpValueEntry
from blackwater.metrics.improvement_factor import improvement_factor, Trial, Problem

from qiskit_aer import AerSimulator, QasmSimulator
from qiskit.providers.fake_provider import FakeMontreal, FakeLima

from torch_geometric.nn import (
    GCNConv,
    TransformerConv,
    GATv2Conv,
    global_mean_pool,
    Linear,
    ChebConv,
    SAGEConv,
    ASAPooling,
    dense_diff_pool,
    avg_pool_neighbor_x
)
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.utils import to_dense_adj, to_dense_batch

from qiskit import QuantumCircuit
from qiskit.circuit.library import U3Gate, CZGate, PhaseGate, CXGate
from mbd_utils import construct_random_clifford, cal_z_exp, calc_imbalance, cal_all_z_exp, construct_mbl_circuit, generate_disorder
from gnn import ExpValCircuitGraphModel

In [17]:
backend = FakeMontreal()
properties = get_backend_properties_v1(backend)

## Local
backend_ideal = QasmSimulator() # Noiseless
backend_noisy = AerSimulator.from_backend(FakeMontreal()) # Noisy

run_config_ideal = {'shots': 10000, 'backend': backend_ideal, 'name': 'ideal'}
run_config_noisy = {'shots': 10000, 'backend': backend_noisy, 'name': 'noisy'}

In [78]:
def construct_tiling(circ_to_append, qbs_with_gates, num_qubit):
    assert len(qbs_with_gates) <= num_qubit
    idle_ind = list(set(range(num_qubit)) - set(qbs_with_gates))
    qr = QuantumRegister(num_qubit)
    cr = ClassicalRegister(num_qubit)
    qc = QuantumCircuit(qr, cr)
    qc.append(circ_to_append, qbs_with_gates, qbs_with_gates)
    qc.measure(idle_ind, idle_ind)
    return qc

In [79]:
qbs_with_gates = [0, 1, 2]
num_qubit = 5
circ_to_append = random_circuit(len(qbs_with_gates), 10, 2, measure=True)
qc = construct_tiling(circ_to_append, qbs_with_gates, num_qubit)
qc.draw(fold=-1)

┌───────────────┐      
q274_0: ┤0              ├──────
        │               │      
q274_1: ┤1              ├──────
        │               │      
q274_2: ┤2              ├──────
        │               │┌─┐   
q274_3: ┤               ├┤M├───
        │  circuit-7225 │└╥┘┌─┐
q274_4: ┤               ├─╫─┤M├
        │               │ ║ └╥┘
 c20_0: ╡0              ╞═╬══╬═
        │               │ ║  ║ 
 c20_1: ╡1              ╞═╬══╬═
        │               │ ║  ║ 
 c20_2: ╡2              ╞═╬══╬═
        └───────────────┘ ║  ║ 
 c20_3: ══════════════════╩══╬═
                             ║ 
 c20_4: ═════════════════════╩═

In [80]:
qc = transpile(qc, backend=backend_noisy, optimization_level=3)
qc.draw(fold=-1, idle_wires=False)

global phase: 0.0010294
                                                                                                                                                                                                                                                                                                                                                                                                                     ░          ┌─┐   
 q274_3 -> 4 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────░──────────┤M├───
             ┌─────────┐ ┌────┐  ┌────────────┐ ┌────┐  ┌─────────┐                  ┌───┐          ┌─────────────┐┌────┐┌──────────────┐┌────┐┌─────────────┐                                                                                                           ┌───┐          ┌─────────────┐ ┌────┐┌─────────────┐┌────┐     ┌─────────────┐     ┌───┐┌────────────────────────┐┌───┐┌───┐ ┌────────────┐ ░       ┌─┐└╥┘   
q274_2 -> 11 ┤ Rz(π/2) ├─┤ √X ├──┤ Rz(-2.213) ├─┤ √X ├──┤ Rz(π/4) ├──────────■───────┤ X ├───────■──┤ Rz(-2.1658) ├┤ √X ├┤ Rz(-0.12806) ├┤ √X ├┤ Rz(-2.4539) ├────────────────────────────────────────────────────────────────────────────────────────────────────■──────┤ X ├───────■──┤ Rz(-1.1843) ├─┤ √X ├┤ Rz(-3.0135) ├┤ √X ├─────┤ Rz(-1.4723) ├─────┤ X ├┤ Rz(-0.147204651905835) ├┤ X ├┤ X ├─┤ Rz(2.9944) ├─░───────┤M├─╫────
             └─────────┘ └────┘┌─┴────────────┴┐├────┤┌─┴─────────┴─┐┌────┐  │  ┌────┴───┴────┐  │  └─────────────┘└────┘└──────────────┘└────┘└─────────────┘    ┌────┐    ┌──────────────┐┌────┐     ┌─────────────┐┌────┐ ┌────────────┐┌────┐┌─────────────┐  │      └───┘       │  └─────────────┘ ├───┬┘└─────────────┘├───┬┘     └─────────────┘     └─┬─┘└────────────────────────┘└─┬─┘└───┘ └────────────┘ ░ ┌─┐   └╥┘ ║    
q274_1 -> 13 ──────────────■───┤ Rz(-0.022434) ├┤ √X ├┤ Rz(-2.3962) ├┤ √X ├──┼──┤ Rz(-3.1393) ├──┼────────────────────────────────────────────────────■───────────┤ √X ├────┤ Rz(-0.74313) ├┤ √X ├──■──┤ Rz(-3.0144) ├┤ √X ├─┤ Rz(-1.579) ├┤ √X ├┤ Rz(-1.5066) ├──┼──────────────────┼──────────────────┤ X ├────────■───────┤ X ├────────────────────────────┼──────────────────────────────┼───────────────────────░─┤M├────╫──╫────
             ┌──────────┐┌─┴─┐ └───┬────────┬──┘└────┘└─────────────┘└────┘┌─┴─┐└┬────────────┤┌─┴─┐ ┌────────────┐┌────┐┌─────────────┐ ┌────┐     ┌─┴─┐     ┌───┴────┴───┐└──────────────┘└────┘┌─┴─┐├─────────────┤├────┤┌┴────────────┤├────┤└┬────────────┤┌─┴─┐┌────────────┐┌─┴─┐┌──────────────┐└─┬─┘      ┌─┴─┐     └─┬─┘ ┌───────────────────────┐  │                              │  ┌────┐               ░ └╥┘┌─┐ ║  ║    
q274_0 -> 14 ┤ Rz(-π/2) ├┤ X ├─────┤ Rz(-π) ├──────────────────────────────┤ X ├─┤ Rz(1.0743) ├┤ X ├─┤ Rz(1.4124) ├┤ √X ├┤ Rz(-1.4754) ├─┤ √X ├─────┤ X ├─────┤ Rz(1.5469) ├──────────────────────┤ X ├┤ Rz(-1.5947) ├┤ √X ├┤ Rz(-1.5006) ├┤ √X ├─┤ Rz(1.7228) ├┤ X ├┤ Rz(1.2696) ├┤ X ├┤ Rz(-0.78725) ├──■────────┤ X ├───────■───┤ Rz(0.147204651905835) ├──■──────────────────────────────■──┤ √X ├───────────────░──╫─┤M├─╫──╫────
             └──────────┘└───┘     └────────┘                              └───┘ └────────────┘└───┘ └────────────┘└────┘└─────────────┘ └────┘     └───┘     └────────────┘                      └───┘└─────────────┘└────┘└─────────────┘└────┘ └────────────┘└───┘└────────────┘└───┘└──────────────┘           └───┘           └───────────────────────┘                                    └────┘               ░  ║ └╥┘ ║  ║ ┌─┐
q274_4 -> 25 ────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────

# Tiling

In [18]:
def construct_mbl_tiling_circuit(qbs_with_gates, total_circuit_size):
    num_spins = 4 # Number of spins. Must be even.
    assert num_spins <= total_circuit_size
    W = 0.8 * np.pi # Disorder strength up to np.pi
    theta = 0.05 * np.pi # Interaction strength up to np.pi
    steps = 6

    disorders = generate_disorder(num_spins, W, seed=0)
    random_mbl_circuit = construct_mbl_circuit(num_spins, disorders, theta, steps)
    qc = construct_tiling(random_mbl_circuit, qbs_with_gates, total_circuit_size)
    return qc, random_mbl_circuit

In [19]:
qc, random_mbl_circuit = construct_mbl_tiling_circuit([0, 1, 2, 3], 8)
qc.draw(fold=-1)

┌───────────────┐ ░ ┌─┐                     
 q54_0: ┤0              ├─░─┤M├─────────────────────
        │               │ ░ └╥┘┌─┐                  
 q54_1: ┤1              ├─░──╫─┤M├──────────────────
        │  circuit-6015 │ ░  ║ └╥┘┌─┐               
 q54_2: ┤2              ├─░──╫──╫─┤M├───────────────
        │               │ ░  ║  ║ └╥┘┌─┐            
 q54_3: ┤3              ├─░──╫──╫──╫─┤M├────────────
        └───────────────┘ ░  ║  ║  ║ └╥┘┌─┐         
 q54_4: ──────────────────░──╫──╫──╫──╫─┤M├─────────
                          ░  ║  ║  ║  ║ └╥┘┌─┐      
 q54_5: ──────────────────░──╫──╫──╫──╫──╫─┤M├──────
                          ░  ║  ║  ║  ║  ║ └╥┘┌─┐   
 q54_6: ──────────────────░──╫──╫──╫──╫──╫──╫─┤M├───
                          ░  ║  ║  ║  ║  ║  ║ └╥┘┌─┐
 q54_7: ──────────────────░──╫──╫──╫──╫──╫──╫──╫─┤M├
                          ░  ║  ║  ║  ║  ║  ║  ║ └╥┘
meas: 8/═════════════════════╩══╩══╩══╩══╩══╩══╩══╩═
                             0  1  2  3  4  5  6  7

In [20]:
transpile(qc, backend=backend_noisy, optimization_level=3)

DAGCircuitError: 'bit mapping invalid: expected 4, got 8'

In [ ]:
from qiskit.transpiler.exceptions import TranspilerError
from scipy.linalg import LinAlgError

def generate_data(generating_func, two_q_gate_depths, save_path: str, n_entries_per_file: int):
    for depth in two_q_gate_depths:
        entries = []

        for i in tqdm_notebook(range(n_entries_per_file)):
            success = False
            while not success:
                try:
                    circuit = generating_func(depth)
                    success = True
                except (LinAlgError, TranspilerError, np.linalg.LinAlgError) as e:
                    print(f"Ran into an error:, {e}")

            job_ideal = execute(circuit, **run_config_ideal)
            job_noisy = execute(circuit, **run_config_noisy)

            counts_ideal = job_ideal.result().get_counts()
            counts_noisy = job_noisy.result().get_counts()

            ideal_exp_val = cal_z_exp(counts_ideal)
            noisy_exp_val = cal_z_exp(counts_noisy)

            graph_data = circuit_to_graph_data_json(
                circuit=circuit,
                properties=properties,
                use_qubit_features=True,
                use_gate_features=True,
            )

            entry = ExpValueEntry(
                circuit_graph=graph_data,
                observable=[],
                ideal_exp_value=ideal_exp_val.tolist(),
                noisy_exp_values=[noisy_exp_val.tolist()],
                circuit_depth=circuit.depth()
            )
            entries.append(entry.to_dict())

        path = f"{save_path}/depth{depth}.json"
        with open(path, "w") as f:
            # pprint(entries)
            json.dump(entries, f)

In [3]:
## Now we need to average over many disorders
num_disorders = 30
qbs_with_gates = [0, 1, 2, 3]

num_spins = 8 # Number of spins. Must be even.
W = 0.8 * np.pi # Disorder strength up to np.pi
theta = 0.05 * np.pi # Interaction strength up to np.pi
steps = 6

even_qubits = np.linspace(0, num_spins, int(num_spins/2), endpoint=False)
odd_qubits = np.linspace(1, num_spins+1, int(num_spins/2), endpoint=False)

# For each disorder realization, make a new disorder
# and compute the charge imbalance using the same physics parameters as before
imbalance_all_ideal = []
imbalance_all_noisy = []
imbalance_all_mitigated = []

for disorder_realization in tqdm_notebook(range(num_disorders)):
    disorders = generate_disorder(num_spins, W, seed=disorder_realization)

        random_mbl_circuit = construct_mbl_circuit(4, disorders, theta, steps)
        qc_list.append()

        job_ideal = execute(circuit, **run_config_ideal)
        job_noisy = execute(circuit, **run_config_noisy)

        counts_ideal = job_ideal.result().get_counts()
        counts_noisy = job_noisy.result().get_counts()

        ideal_exp_val = cal_z_exp(counts_ideal)
        noisy_exp_val = cal_z_exp(counts_noisy)

        graph_data = circuit_to_graph_data_json(
            circuit=circuit,
            properties=properties,
            use_qubit_features=True,
            use_gate_features=True,
        )

        entry = ExpValueEntry(
            circuit_graph=graph_data,
            observable=[],
            ideal_exp_value=ideal_exp_val.tolist(),
            noisy_exp_values=[noisy_exp_val.tolist()],
            circuit_depth=circuit.depth()
        )
        entries.append(entry.to_dict())

    path = f"{save_path}/depth{depth}.json"
    with open(path, "w") as f:
        # pprint(entries)
        json.dump(entries, f)

  0%|          | 0/30 [00:00<?, ?it/s]

NameError: name 'model' is not defined